In [24]:
#TESTING WEB SCRAPING

#For single recipe, taking nutrient info

In [25]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [26]:
url = 'https://www.allrecipes.com/recipe/18057/sweet-potato-casserole-ii/'
url1 = url + 'fullrecipenutrition'
r = requests.get(url1)
print('Response body: {0}'.format(r.text))

Response body: 
<div class="recipe-nutrition">
    <h1 class="nutrition_h1">Nutritional Information</h1>
    <h2>Sweet Potato Casserole II</h2>
    <div class="nutrition-top light-underline">

        Servings Per Recipe: 16<br />
        <span class="semi-bold">Amount Per Serving</span><br />
            <span class="semi-bold">Calories:</span> 309<br />
    </div>
    <div class="nutrition-body-label">% Daily Value *</div>
    <div class="nutrition-body">
        <div class="nutrition-row">
            <span class="nutrient-name">Total Fat: <span aria-label="15.3 grams"></span><span class="nutrient-value" aria-hidden="true">15.3g</span></span>
                <span class="daily-value">24 %</span>
        </div>
                <div class="nutrition-row">
            <span class="nutrient-name indent">Saturated Fat: <span aria-label="7.0 grams"></span><span class="nutrient-value" aria-hidden="true">7.0g</span></span>
        </div>
                <div class="nutrition-row">
         

In [27]:
page_body = r.text
soup = BeautifulSoup(page_body, 'html.parser')

In [28]:
#this is the div containing all the nutrition information
nutrition_info = soup.find_all('div', class_='nutrition-row')

In [29]:
#nutrition_info

In [30]:
nutrient_list = []
recipe = soup.find('h2').text
for n in nutrition_info:
    name = n.find(class_ = 'nutrient-name').text
    amount = n.find(class_ = 'nutrient-value').text
    name = name[:name.index(':')]
    nutrient_list.append({'nutrient': name, 'amount': amount, 'recipe': recipe, 'URL': url})
nutrients = pd.DataFrame(nutrient_list)

nutrients.head()

,URL,amount,nutrient,recipe
0,https://www.allrecipes.com/recipe/18057/sweet-potato-casserole-ii/,15.3g,Total Fat,Sweet Potato Casserole II
1,https://www.allrecipes.com/recipe/18057/sweet-potato-casserole-ii/,7.0g,Saturated Fat,Sweet Potato Casserole II
2,https://www.allrecipes.com/recipe/18057/sweet-potato-casserole-ii/,49mg,Cholesterol,Sweet Potato Casserole II
3,https://www.allrecipes.com/recipe/18057/sweet-potato-casserole-ii/,103mg,Sodium,Sweet Potato Casserole II
4,https://www.allrecipes.com/recipe/18057/sweet-potato-casserole-ii/,337mg,Potassium,Sweet Potato Casserole II


In [31]:
# & ingredients

In [32]:
s = requests.get(url)

ConnectionError: ('Connection aborted.', OSError("(104, 'ECONNRESET')",))

In [ ]:
page_body2 = s.text
soup2 = BeautifulSoup(page_body2, 'html.parser')

In [ ]:
ingredient_info = soup2.find_all(class_='recipe-container-outer')
#ingredient_info

In [ ]:
ingredient_list = []
for i in ingredient_info:
    ingredients = i.find_all('span', {'itemprop':'recipeIngredient'}, class_ = 'recipe-ingred_txt added')
    for x in ingredients:
        ingredient_list.append({'ingredient': x.text, 'URL':url})

ingredients = pd.DataFrame(ingredient_list)

ingredients.head()

In [34]:
# FUNCTION FOR WEB SCRAPING

import time
def get_recipe_data(url, nutrient_df, ingredient_df):
    """
    Scrape allrecipes.com for information on the nutrient content and ingredients of each recipe
    Based on the url, ingredients and nutrient content are extracted separately from the website.
    
    IN: url: URL of the specific recipe as a string
    OUT: nutrient and ingredient dataframes with the information for the given recipe
    """
    
    #nutrition information
    url_nutrition = url + 'fullrecipenutrition'
    r1 = requests.get(url_nutrition)
    time.sleep(1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    
    #this is the div containing all the nutrition information
    nutrition_info = soup1.find_all('div', class_='nutrition-row')
    
    nutrient_list = []
    recipe = soup1.find('h2').text
    for n in nutrition_info:
        name = n.find(class_ = 'nutrient-name').text
        amount = n.find(class_ = 'nutrient-value').text
        name = name[:name.index(':')]
        nutrient_list.append({'nutrient': name, 'amount': amount, 'recipe': recipe, 'URL': url})
    
    nutrients = pd.DataFrame(nutrient_list)
    nutrient_df = nutrient_df.append(nutrients, sort=True)
    
    #ingredients
    r2 = requests.get(url)
    time.sleep(1)
    soup2 = BeautifulSoup(r2.text, 'html.parser')
    ingredient_info = soup2.find_all(class_='recipe-container-outer')
    
    ingredient_list = []
    for i in ingredient_info:
        ingredient = i.find_all('span', {'itemprop':'recipeIngredient'}, class_ = 'recipe-ingred_txt added')
        for x in ingredient:
            ingredient_list.append({'ingredient': x.text, 'URL':url})

    ingredients = pd.DataFrame(ingredient_list)
    ingredient_df = ingredient_df.append(ingredients, sort=True)
    
    return nutrient_df, ingredient_df

In [ ]:
#test the function
#url = 'https://www.allrecipes.com/recipe/18057/sweet-potato-casserole-ii/'
#nutrient_columns = ['URL', 'amount', 'nutrient', 'recipe']
#ingredient_columns = ['URL', 'ingredient']
#nutrient_df = pd.DataFrame(columns=nutrient_columns)
#ingredient_df = pd.DataFrame(columns=ingredient_columns)

#nutrient_df, ingredient_df = get_recipe_data(url, nutrient_df, ingredient_df)

In [ ]:
#url2 = 'https://www.allrecipes.com/recipe/7589/allspice-cream-cheese-frosting/'
#nutrient_df, ingredient_df = get_recipe_data(url2, nutrient_df, ingredient_df)

In [ ]:
#nutrient_df

In [35]:
dinner_url_df = pd.read_csv('data/recipeLists/dinner_url.csv')
dinner_url_df.columns = ["originGridUrl", "recipeName", "recipeURL"]
pd.options.display.max_colwidth = 200
#display(dinner_url_df['recipeURL'].head())

In [36]:
#dinner_url_df['recipeURL'] = dinner_url_df['recipeURL'].split("?")[0]
dinner_url_df['recipeURL'] = dinner_url_df['recipeURL'].astype(str).apply(lambda x: x.split("?")[0])
#dinner_url_df['recipeURLPrime'] = str(dinner_url_df['recipeURL']).split("?")[0]
pd.set_option('display.max_rows', None,'display.max_columns', None)
display(dinner_url_df.head())

,originGridUrl,recipeName,recipeURL
0,https://www.allrecipes.com/recipes/17562/dinner/?page=1,Smothered Chicken Breasts,https://www.allrecipes.com/recipe/222607/smothered-chicken-breasts/
1,https://www.allrecipes.com/recipes/17562/dinner/?page=1,Asian Beef with Snow Peas,https://www.allrecipes.com/recipe/15679/asian-beef-with-snow-peas/
2,https://www.allrecipes.com/recipes/17562/dinner/?page=1,Pasta Pomodoro,https://www.allrecipes.com/recipe/23847/pasta-pomodoro/
3,https://www.allrecipes.com/recipes/17562/dinner/?page=1,Fry Bread Tacos II,https://www.allrecipes.com/recipe/50435/fry-bread-tacos-ii/
4,https://www.allrecipes.com/recipes/17562/dinner/?page=1,Pork Marsala,https://www.allrecipes.com/recipe/140829/pork-marsala/


In [37]:
dinnerURLList = dinner_url_df.recipeURL.tolist()

In [38]:
type(dinnerURLList)

list

In [39]:
nutrient_columns2 = ['URL', 'amount', 'nutrient', 'recipe']
ingredient_columns2 = ['URL', 'ingredient']
nutrient_df2 = pd.DataFrame(columns=nutrient_columns2)
ingredient_df2 = pd.DataFrame(columns=ingredient_columns2)

import time
for url in dinnerURLList[:100]:
    nutrient_df2, ingredient_df2 = get_recipe_data(url, nutrient_df2, ingredient_df2)
    time.sleep(1)
    

ConnectionError: ('Connection aborted.', OSError("(104, 'ECONNRESET')",))

In [ ]:
nutrient_df2

In [ ]:
nutrient_df2.shape